In [115]:
#libraries
import pandas as pd
import numpy as np
import datetime
import os
import getpass
import requests
import pyproj
#import required libraries
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
from osgeo import gdal
import xml.etree.ElementTree as ET

## Uwaga
Skrypt nie jest reprodukowalny:

- pobieranie granic działek jest niestabilne
- stąd ręczne indeksowanie wycinania się zmienia
- backup ramek ze współrzędnymi w plikach csv


## Reading sheets with lots

In [186]:
#preprocessing of sheets with found lots
def preprocess_csv_coordinates_data(csv_path: str, id_column_name: str,
                                    coordinates_column_name: str, date_column_name: str) -> pd.DataFrame:
    
    coord_data = pd.read_csv(csv_path) 
    coord_data = coord_data[[coordinates_column_name, id_column_name, date_column_name]]
    splitted_coords = coord_data[coordinates_column_name].str.split(" ", n = 1, expand = True)
    
    coord_data["date_from"] = pd.to_datetime(coord_data[date_column_name]) - datetime.timedelta(days=2)
    coord_data["date_to"] = pd.to_datetime(coord_data[date_column_name]) + datetime.timedelta(days=2)
    
    
    coord_data["id"] = coord_data[coord_data.columns[1]]
    #pattern = r"\.|\/"
    #coord_data["id"] = coord_data["id"].str.replace(pattern, "_")


    coord_data["latitude"] = splitted_coords[0]
    coord_data["latitude"] = coord_data["latitude"].str.strip(",")
    coord_data["latitude"] = coord_data["latitude"].astype("float")


    coord_data["longitude"] = splitted_coords[1]
    coord_data["longitude"] = coord_data["longitude"].str.strip(",")
    coord_data["longitude"] = coord_data["longitude"].astype("float")

    coord_data = coord_data[["id", "latitude", "longitude", "date_from", "date_to"]]
    return coord_data

#reading data from sheets
#change if doesnt work
path_rzepak_csv =  "../data/dzialki_rzepakowe.csv"
path_nierzepak_csv = "../data/dzialki_bezrzepaku.csv"

coord_data_rzepak = preprocess_csv_coordinates_data(path_rzepak_csv, "identyfikator działki", "współrzędne", "data sentinel")
coord_data_nierzepak = preprocess_csv_coordinates_data(path_nierzepak_csv, "identyfikator działki", "współrzędne", "data sentinel")

## Adding lot border data

In [190]:
def cord_reader(plot_ids_list):
    
    # Taking list contaings plots ID as strings 
    # example ["260101_5.0037.569"]
    
    # Return dictionary with plots ID as key and 
    # cords of plot as value 
    
    #data check 
   
    if( not isinstance(plot_ids_list,list)): 
        raise Exception("List object is required")
    if( not isinstance(plot_ids_list[0],str)):
        raise Exception("List didnt contains only strings")
        
    list_of_cords= ["empty"]*len(plot_ids_list)
    iterator = 0
    s = requests.Session()
    for i in plot_ids_list:
        # ganerated with https://curl.trillworks.com/ 
     
                    


        cookies = {
        '_ga': 'GA1.2.917990734.1605779206',
        '_gid': 'GA1.2.1851729915.1607357978',
        'PHPSESSID': 'u6rcvr3mu7vcai1hm2e0tj8cm5',
        '_gat_gtag_UA_122392520_1': '1',
                }

        headers = {
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'Content-type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://polska.e-mapa.net',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://polska.e-mapa.net/',
        'Accept-Language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
            }

        data = {
        'action': 'wsporzedne_granic',
        'id_dzialki': i,
        'srid': '2180'
        }



        response = s.post('https://polska.e-mapa.net/application/modules/dzus/dzus.php', headers=headers, cookies=cookies, data=data)

        out  = response.text
        # OUT in form of string 
        # Spliting and extracting cords 
        out = out.split("'")
        out = out[1:len(out)-1]
        final =[]
        for i in out:
            
            if(i>"-"):
                final.append(i)
        
        list_of_cords[iterator] = final
        iterator = iterator +1
        
        
    
    res = dict(zip(plot_ids_list, list_of_cords))
        
    return res

### Rzepak

In [191]:
#empty column for rectangle and border coords

#coord_data_rzepak["border"] = np.nan

#using only rect for cutting
coord_data_rzepak["xmin"] = np.nan
coord_data_rzepak["ymin"] = np.nan
coord_data_rzepak["xmax"] = np.nan
coord_data_rzepak["ymax"] = np.nan

input_proj = pyproj.Proj(init='epsg:2180')
output_proj = pyproj.Proj(init="epsg:4326")


for i in range(len(coord_data_rzepak)):
    
    #lot ID
    lot_id = coord_data_rzepak.iloc[i, 0]
    
    #reading coords of lot
    coord_dict = cord_reader([lot_id])
    coord_puwg = coord_dict.get(lot_id)
    
    print(coord_dict)
    
    xmin = 1000
    ymin = 1000
    xmax = 0
    ymax = 0

    for j in range(len(coord_puwg)):
        coord_puwg[j] = str.split(coord_puwg[j])

        y_in = float(coord_puwg[j][0])
        x_in = float(coord_puwg[j][1])

        x_out, y_out = pyproj.transform(input_proj, output_proj, x_in, y_in)

        coord_puwg[j][0] = x_out
        coord_puwg[j][1] = y_out

        if(x_out<xmin):
            xmin = x_out
        elif(x_out>xmax):
            xmax = x_out
        if(y_out<ymin):
            ymin = y_out
        elif(y_out>ymax):
            ymax = y_out
    
    coord_data_rzepak.loc[i, "xmin"] = xmin
    coord_data_rzepak.loc[i, "xmax"] = xmax
    coord_data_rzepak.loc[i, "ymin"] = ymin
    coord_data_rzepak.loc[i, "ymax"] = ymax

/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

{'101507_2.0033.187': ['443304.8 581671.7', '443583.8 581705.0', '443565.6 581555.8', '443305.3 581525.4', '443304.8 581671.7']}


/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


{'101507_2.0033.248': ['442961.6 581675.3', '443067.6 581593.6', '442868.7 581373.3', '442798.0 581427.2', '442759.8 581455.2', '442961.6 581675.3']}
{'101507_2.0033.261': ['442222.1 582442.4', '442421.8 582676.5', '442501.3 582654.5', '442283.9 582397.1', '442222.1 582442.4']}
{'101507_2.0018.109': ['444790.2 586578.3', '444471.0 586578.1', '444472.7 586658.2', '444792.4 586658.6', '444790.7 586596.0', '444790.2 586578.3']}
{'121304_5.0013.401': []}
{'121304_5.0013.385': []}
{'121304_5.0013.503/11': []}
{'121304_5.0011.571': []}
{'121304_5.0011.466': []}
{'120906_2.0003.2761/2': ['227138.6 571395.9', '227159.7 571392.1', '227179.3 571385.3', '227205.7 571377.4', '227270.1 571360.6', '227329.9 571345.7', '227368.3 571337.3', '227393.0 571330.5', '227415.2 571321.9', '227408.7 571150.4', '227407.1 571055.3', '227366.3 571040.0', '227363.1 571039.4', '227337.0 571038.7', '227333.7 571026.2', '227303.6 571030.1', '227296.1 571017.5', '227280.6 571006.0', '227255.9 571005.7', '227251.5 570

{'121902_2.0002.729': ['232093.1 590516.0', '232094.7 590511.7', '232100.6 590491.2', '232103.4 590475.0', '232106.2 590454.4', '232108.7 590431.4', '232113.3 590404.8', '232114.6 590399.1', '232115.5 590395.5', '232122.0 590379.6', '232131.1 590354.4', '232135.2 590340.1', '232130.6 590338.4', '232124.7 590338.5', '232022.8 590343.8', '231998.8 590344.6', '231952.5 590346.1', '231846.3 590349.1', '231741.6 590351.4', '231692.5 590352.5', '231695.0 590389.4', '231698.3 590406.3', '231702.8 590419.7', '231717.4 590452.2', '231725.9 590474.2', '231740.1 590516.2', '231743.6 590526.4', '231818.0 590514.3', '231906.3 590502.7', '231937.2 590499.3', '231936.3 590484.4', '231955.2 590488.3', '231996.2 590496.7', '232093.1 590516.0']}
{'260704_5.0010.16': ['336592.5 674598.0', '336647.5 674639.3', '336710.2 674698.2', '336756.6 674755.0', '336378.9 674777.4', '336424.2 674610.0', '336530.8 674602.4', '336592.5 674598.0']}
{'260704_5.0003.1035': ['338434.0 675178.3', '338403.4 675177.6', '3383

{'021804_5.0016.181/2': ['367617.5 329068.7', '367645.6 329080.5', '367858.2 329302.3', '367873.6 329285.1', '367890.9 329251.8', '367924.9 329216.5', '367966.6 329163.8', '368070.7 329155.4', '368221.6 329136.2', '368243.3 329120.4', '368341.6 329002.0', '368359.6 328919.0', '368462.7 328922.5', '368456.5 328788.2', '368453.8 328736.3', '368401.3 328618.3', '368380.2 328571.4', '367815.5 328853.6', '367613.6 329014.6', '367607.1 329031.5', '367617.7 329060.3', '367617.5 329068.7']}
{'021804_5.0002.365/2': ['366776.2 326780.8', '366774.0 326755.3', '366739.4 326634.1', '366732.5 326615.6', '366711.8 326556.9', '366682.8 326477.7', '366662.8 326418.9', '366650.2 326378.6', '366637.7 326339.9', '366306.1 326386.6', '366369.4 326835.5', '366392.1 326831.8', '366481.1 326822.6', '366580.0 326809.1', '366678.7 326795.9', '366760.8 326785.1', '366776.2 326780.8']}
{'021804_5.0002.196': ['364689.9 327217.3', '365031.0 327266.6', '365010.8 327184.2', '364744.0 327131.4', '364728.8 327145.8', '

{'300308_2.0013.60/1': ['507834.4 409120.3', '507805.2 409147.6', '507848.0 409193.4', '507821.6 409219.1', '507801.1 409238.1', '507800.5 409237.6', '507769.1 409208.3', '507740.0 409246.1', '507729.3 409237.7', '507727.7 409236.4', '507705.3 409262.7', '507684.1 409287.4', '507684.5 409287.6', '507685.6 409288.4', '507708.6 409306.2', '507728.6 409321.5', '507746.1 409314.1', '507742.9 409337.3', '507740.6 409368.6', '507741.8 409383.6', '507746.3 409398.4', '507777.1 409494.0', '507784.1 409514.0', '507785.5 409527.3', '507785.0 409538.2', '507777.7 409559.5', '507764.3 409588.3', '507730.4 409654.3', '507710.3 409645.4', '507591.4 409595.1', '507508.1 409559.1', '507405.3 409408.5', '507415.0 409414.0', '507529.4 409211.8', '507531.5 409209.4', '507246.4 408916.7', '507453.1 408707.8', '507834.4 409120.3', '507437.4 409416.9', '507442.2 409418.9', '507549.2 409234.2', '507544.7 409230.9', '507437.4 409416.9']}
{'300308_2.0007.49': ['510443.4 406176.1', '510676.6 406144.8', '510695.

{'281702_2.0013.79': ['654655.7 635203.4', '654643.7 635196.1', '654634.9 635192.6', '654632.1 635189.8', '654611.6 635173.8', '654604.9 635164.5', '654598.9 635152.7', '654594.8 635141.7', '654592.5 635120.7', '654600.9 635103.8', '654604.2 635100.8', '654606.5 635072.3', '654603.5 635061.8', '654601.1 635053.5', '654601.8 635044.8', '654603.8 635034.3', '654602.6 635020.8', '654608.8 635020.2', '654595.4 635007.8', '654590.1 634982.4', '654585.1 634952.8', '654580.6 634930.2', '654575.0 634918.8', '654580.0 634899.4', '654575.6 634866.9', '654568.8 634865.3', '654548.9 634848.6', '654532.4 634827.8', '654518.8 634812.8', '654505.6 634797.1', '654465.1 634772.1', '654448.5 634760.6', '654437.7 634752.3', '654435.5 634750.7', '654423.1 634772.1', '654418.2 634786.3', '654414.4 634841.5', '654406.0 634961.6', '654372.4 634978.2', '654322.4 635003.9', '654177.6 635077.4', '654229.0 635241.1', '654224.7 635270.1', '654188.4 635398.4', '654139.3 635550.2', '654319.0 635545.1', '654378.2 63

{'280302_2.0012.18/6': ['606663.5 577518.9', '606688.5 577508.3', '606706.3 577517.3', '606730.1 577539.7', '606787.8 577532.1', '606853.7 577517.5', '606934.0 577522.1', '606973.4 577524.0', '606991.2 577519.8', '607010.0 577510.9', '607026.3 577504.8', '607061.0 577494.7', '607089.1 577486.6', '607114.0 577472.1', '607129.5 577455.4', '607165.1 577408.3', '607190.0 577377.9', '607220.4 577341.4', '607260.7 577293.6', '607044.8 577185.7', '606940.2 577149.8', '606881.5 577144.1', '606836.4 577158.4', '606801.5 577146.6', '606644.2 577068.8', '606636.2 577081.0', '606601.5 577134.2', '606567.7 577185.2', '606526.7 577235.4', '606519.8 577244.9', '606440.5 577346.4', '606427.7 577364.1', '606444.3 577379.5', '606390.0 577451.8', '606364.5 577444.8', '606317.0 577513.4', '606257.5 577607.7', '606226.3 577667.1', '606372.2 577635.5', '606454.6 577620.6', '606533.2 577594.3', '606591.6 577576.0', '606579.2 577549.0', '606632.7 577517.7', '606663.5 577518.9']}
{'280302_2.0012.167/2': ['6087

{'281103_2.0014.27': ['610193.8 582138.5', '610203.1 582194.4', '610544.6 582327.1', '610670.3 582375.3', '610595.0 582265.8', '610541.9 582246.8', '610517.9 582236.4', '610462.0 582208.4', '610396.0 582184.4', '610285.8 582144.7', '610212.1 582115.6', '610188.3 582103.7', '610189.6 582111.9', '610193.8 582138.5']}
{'281103_2.0014.20/6': ['611279.7 582506.4', '611247.9 582495.1', '611152.6 582461.2', '610988.7 582403.0', '610966.2 582423.8', '611005.4 582418.3', '611027.8 582491.8', '611066.3 582477.9', '611081.2 582520.8', '611040.8 582534.9', '611043.0 582542.4', '610812.5 582571.4', '610869.9 582653.1', '610965.9 582794.3', '610980.1 582816.7', '611128.9 582867.2', '611269.9 582914.9', '611315.7 582930.4', '611420.8 582965.9', '611550.5 583009.7', '611551.9 583006.0', '611571.7 583012.9', '611594.6 583081.1', '611603.2 583055.3', '611627.3 583023.0', '611679.5 582955.0', '611763.7 582938.8', '611847.7 582921.1', '611947.6 582846.2', '612032.9 582756.0', '611879.5 582705.0', '611757.

### Nierzepak

In [202]:
#empty column for rectangle and border coords

#coord_data_rzepak["border"] = np.nan

#using only rect for cutting
coord_data_nierzepak["xmin"] = np.nan
coord_data_nierzepak["ymin"] = np.nan
coord_data_nierzepak["xmax"] = np.nan
coord_data_nierzepak["ymax"] = np.nan

input_proj = pyproj.Proj(init='epsg:2180')
output_proj = pyproj.Proj(init="epsg:4326")


for i in range(len(coord_data_nierzepak)):
    
    #lot ID
    lot_id = coord_data_nierzepak.iloc[i, 0]
    
    #reading coords of lot
    coord_dict = cord_reader([lot_id])
    coord_puwg = coord_dict.get(lot_id)
    
    print(coord_dict)
    
    xmin = 1000
    ymin = 1000
    xmax = 0
    ymax = 0

    for j in range(len(coord_puwg)):
        coord_puwg[j] = str.split(coord_puwg[j])

        y_in = float(coord_puwg[j][0])
        x_in = float(coord_puwg[j][1])

        x_out, y_out = pyproj.transform(input_proj, output_proj, x_in, y_in)

        coord_puwg[j][0] = x_out
        coord_puwg[j][1] = y_out

        if(x_out<xmin):
            xmin = x_out
        elif(x_out>xmax):
            xmax = x_out
        if(y_out<ymin):
            ymin = y_out
        elif(y_out>ymax):
            ymax = y_out
    
    coord_data_nierzepak.loc[i, "xmin"] = xmin
    coord_data_nierzepak.loc[i, "xmax"] = xmax
    coord_data_nierzepak.loc[i, "ymin"] = ymin
    coord_data_nierzepak.loc[i, "ymax"] = ymax

/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/piotr/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

{'240504_2.0005.AR_1.726/307': ['260043.2 469141.0', '260049.1 469133.4', '260040.0 469123.7', '260049.2 469121.7', '260038.9 469109.1', '260045.0 469105.2', '260056.3 469117.8', '260066.7 469107.9', '260047.2 469086.0', '260035.5 469072.8', '260023.3 469083.9', '259941.2 468996.5', '259929.1 469003.4', '259906.5 469014.4', '259893.5 469003.5', '259881.0 469008.5', '259857.4 469017.9', '259885.1 469045.0', '259916.6 469075.9', '259933.1 469092.1', '259949.6 469108.2', '259966.0 469124.3', '259970.5 469128.7', '259974.3 469125.0', '259989.5 469111.5', '259990.4 469112.4', '260008.8 469097.7', '260033.2 469125.1', '260030.3 469127.5', '260043.2 469141.0']}


/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


{'240504_2.0005.AR_11.151/27': ['259707.1 468514.1', '259703.3 468514.4', '259677.8 468517.0', '259606.5 468528.1', '259584.3 468545.6', '259657.4 468628.8', '259719.0 468697.7', '259791.4 468617.0', '259811.7 468594.9', '259774.4 468566.9', '259751.1 468549.5', '259707.1 468514.1']}
{'240504_2.0006.AR_3.290/60': ['260595.3 465783.5', '260568.8 465791.3', '260524.8 465803.1', '260481.0 465814.8', '260401.6 465834.1', '260357.6 465844.8', '260325.9 465852.5', '260327.4 465864.1', '260327.1 465881.9', '260327.1 465884.0', '260329.7 465923.2', '260331.8 465943.0', '260368.6 465935.2', '260366.0 466024.8', '260384.2 466019.3', '260575.8 466010.7', '260609.3 466013.5', '260613.2 466014.0', '260635.6 466018.6', '260658.3 466023.0', '260683.2 466023.9', '260721.5 466025.2', '260757.9 466023.9', '260832.2 466017.6', '260816.5 465908.9', '260811.6 465888.5', '260803.7 465867.7', '260798.4 465859.4', '260790.8 465850.7', '260777.2 465840.6', '260750.9 465829.5', '260721.0 465818.5', '260709.0 46

{'140607_5.0014.79/7': ['427456.0 620072.3', '427429.4 620050.3', '427453.3 620019.0', '427481.9 619981.5', '427428.2 619938.3', '427479.3 619872.0', '427516.1 619824.3', '427565.6 619760.2', '427559.8 619756.1', '427548.5 619748.0', '427455.8 619677.5', '427454.6 619676.6', '427312.9 619799.1', '427227.3 619872.8', '427171.0 619915.0', '427168.2 619918.7', '427109.5 619998.0', '427091.7 620022.5', '427091.9 620023.0', '427094.4 620029.1', '427140.7 620090.4', '427149.5 620101.5', '427153.6 620110.6', '427179.7 620168.2', '427188.6 620187.8', '427201.6 620211.5', '427206.2 620216.8', '427233.6 620251.0', '427251.7 620270.6', '427273.1 620291.5', '427280.4 620299.7', '427427.6 620109.1', '427456.0 620072.3']}
{'140607_5.0028.218': ['423793.5 620610.2', '423790.0 620605.6', '423732.5 620530.6', '423721.0 620513.2', '423669.4 620443.6', '423638.1 620403.7', '423522.0 620544.6', '423474.2 620592.7', '423463.9 620602.0', '423450.1 620612.5', '423434.4 620623.2', '423420.2 620631.3', '423408

{'320407_5.0005.492/4': ['656475.7 207018.7', '656074.6 207171.7', '656214.0 207714.3', '656755.5 207585.3', '656850.0 207658.4', '656852.3 207650.7', '656762.7 207581.5', '656653.8 207531.0', '656572.3 207490.7', '656567.6 207481.8', '656640.5 207207.7', '656646.9 207188.2', '656664.1 207123.6', '656663.4 207119.1', '656630.4 207077.6', '656608.0 207065.3', '656570.8 207044.2', '656563.1 207035.9', '656475.7 207018.7']}
{'320407_5.0002.12/5': ['661782.3 207032.5', '661774.0 207154.6', '661789.4 207163.7', '661733.6 207381.0', '661752.6 207385.9', '662025.5 207457.2', '662308.6 207530.0', '662347.4 207542.8', '662421.1 207581.6', '662444.7 207602.2', '662458.4 207616.5', '662525.2 207693.9', '662592.2 207762.9', '662711.1 207891.1', '662732.5 207904.1', '662803.4 207939.2', '662902.6 207988.8', '663126.0 208044.1', '663296.9 208079.3', '663316.2 208072.0', '663333.0 208066.6', '663402.0 208025.5', '663437.7 207989.0', '663438.1 207984.2', '663481.3 207923.9', '663484.6 207916.8', '6634

### Filtering to lots with scraped borders

In [203]:
coord_data_rzepak = coord_data_rzepak[coord_data_rzepak["xmax"] > 0]
coord_data_nierzepak = coord_data_nierzepak[coord_data_nierzepak["xmax"] > 0]

In [204]:
len(coord_data_rzepak)

93

In [205]:
len(coord_data_nierzepak)

41

In [201]:
coord_data_rzepak

,id,latitude,longitude,date_from,date_to,xmin,ymin,xmax,ymax
0,101507_2_0033_187,51.851296,20.185202,2020-05-15,2020-05-19,20.183968,51.850081,20.186642,51.852585
1,101507_2_0033_248,51.846549,20.183986,2020-05-15,2020-05-19,20.181657,51.845212,20.186063,51.847959
2,101507_2_0033_261,51.841394,20.198399,2020-05-15,2020-05-19,20.196381,51.840232,20.200470,51.842711
3,101507_2_0018_109,51.861189,20.258306,2020-05-15,2020-05-19,20.257621,51.859827,20.258870,51.862704
9,120906_2.0003.2761/2,49.908453,19.991767,2020-05-15,2020-05-19,19.988927,49.907083,19.994613,49.909794
...,...,...,...,...,...,...,...,...,...
95,281103_2.0013.14/4,53.305840,20.336440,2020-05-20,2020-05-24,20.328410,53.303232,20.350255,53.309831
96,281103_2.0029.15/8,53.309071,20.339830,2020-05-20,2020-05-24,20.332178,53.303232,20.356925,53.311104
97,281103_2.0013.14/3,53.310456,20.343735,2020-05-20,2020-05-24,20.336447,53.305735,20.352844,53.315063
98,281104_5.0019.213,53.350618,20.372911,2020-05-20,2020-05-24,20.370589,53.347151,20.375958,53.353542


In [206]:
coord_data_nierzepak

,id,latitude,longitude,date_from,date_to,xmin,ymin,xmax,ymax
0,240504_2.0005.AR_1.726/307,50.205773,18.566558,2020-04-10,2020-04-14,18.565406,50.205111,18.567423,50.206999
1,240504_2.0005.AR_11.151/27,50.202685,18.561080,2020-04-10,2020-04-14,18.558664,50.202629,18.561236,50.204678
2,240504_2.0006.AR_3.290/60,50.212573,18.521894,2020-04-10,2020-04-14,18.520310,50.209151,18.523714,50.213716
3,240506_5.0001.AR_11.41,50.215569,18.482311,2020-04-10,2020-04-14,18.472054,50.211661,18.476978,50.215561
5,241609_2.0006.AR_7.1091,50.570401,19.485375,2020-04-10,2020-04-14,19.481486,50.569062,19.500053,50.570726
6,241609_2.0008.AR_1.25,50.558658,19.466328,2020-04-10,2020-04-14,19.464426,50.558165,19.466564,50.559274
7,241604_2.0008.AR_3.336,50.571328,19.525360,2020-04-10,2020-04-14,19.524049,50.571046,19.528180,50.572571
8,241604_2.0008.AR_15.1854/6,50.569891,19.536670,2020-04-10,2020-04-14,19.535690,50.568565,19.538563,50.570915
9,240504_2.0007.AR_9.75/2,50.201721,18.581408,2020-04-10,2020-04-14,18.578948,50.199926,18.585433,50.203800
15,200203_2.0003.886,53.184867,23.026001,2020-05-12,2020-05-16,23.025059,53.183880,23.027601,53.184959


In [207]:
coord_data_rzepak.to_csv("~/Pulpit/rzepak_back.csv")
coord_data_nierzepak.to_csv("~/Pulpit/nierzepak_back.csv")

# Cutting to jpg

In [208]:
def cut_plot(photo_dir_path, output_path, xmin, ymin, xmax, ymax, epsg_info):
    """
    photo_dir_path: sciezka do katalogu _extracted
    output_path: sciezka pliku wycietego zdjecia np.: zdjecie.png
    xmin, ymin, xmax, ymax: wspolrzedne prostokata do wyciecia
    """
    
    #target_dir = os.listdir(photo_dir_path + "/GRANULE/")[0]
    
    #XML part
    #root = ET.parse(photo_dir_path + "/MTD_TL.xml").getroot()
    #epsg_info = list(list(root)[1])[0].find("HORIZONTAL_CS_CODE").text[5:]
    
    
    #photos_paths = os.listdir(photo_dir_path)
    #for f in photos_paths:
    #    if(f.find("TCI") != -1):
    #        photo_path = photo_dir_path + f
    #        break
    
    #JP2 to TIFF
    #gdal.Translate("temp_tiff.tiff", photo_path)
    
    #Wycinanie opcje
    opt = gdal.WarpOptions(srcSRS = 'epsg:'+epsg_info, dstSRS = 'epsg:4326', 
                       outputBounds = (xmin, ymin, xmax, ymax), format = "Gtiff")
    
    #Wycinanie
    res = gdal.Warp('usunac.tiff', 'temp_tiff.tiff', options = opt)
    
    #Wyciete do png
    gdal.Translate(output_path, res)
    
    #Usuwanie temp_tiff
    os.remove("usunac.tiff")
    os.remove(output_path + ".aux.xml")
    
    del res
    
    return

In [209]:
def cut_few_lots(coord_df, extracted_dir_path, out_dir_path):
    
    pattern = r"\.|\/"
    coord_df["id"] = coord_df["id"].str.replace(pattern, "_")
    
    #target_dir = os.listdir(photo_dir_path + "/GRANULE/")[0]
    
    #XML part
    root = ET.parse(extracted_dir_path + "/MTD_TL.xml").getroot()
    cur_epsg_info = list(list(root)[1])[0].find("HORIZONTAL_CS_CODE").text[5:]
    
    
    photos_paths = os.listdir(extracted_dir_path)
    for f in photos_paths:
        if(f.find("TCI") != -1):
            photo_path = extracted_dir_path + f
            break
    
    #JP2 to TIFF
    gdal.Translate("temp_tiff.tiff", photo_path)
    
    for i in range(len(coord_df)):
        
        pattern = r"\.|\/"
        out_path = out_dir_path + coord_df.loc[coord_df.index[i], "id"] + ".jpg"
        
        cut_plot(extracted_dir_path, 
                out_path,
                xmin = coord_df.loc[coord_df.index[i], "xmin"],
                ymin = coord_df.loc[coord_df.index[i], "ymin"],
                xmax = coord_df.loc[coord_df.index[i], "xmax"],
                ymax = coord_df.loc[coord_df.index[i], "ymax"],
                epsg_info = cur_epsg_info)
    
    os.remove("temp_tiff.tiff")
    return

## Rzepak

In [197]:
os.listdir("download_rzepak")

['021804_5.0002.196_extracted',
 '060103_2.0002.283_extracted',
 '121304_5.0013.401_extracted',
 '101507_2.0033.187_extracted',
 '281408_2.0010.116_extracted',
 '181004_2.0003.1471_extracted',
 '260704_5.0010.16_extracted',
 '281702_2.0013.79_extracted',
 '300308_2_0015_62_extracted']

In [198]:
#łódzkie kilka
cut_few_lots(coord_data_rzepak.iloc[0:4],
            extracted_dir_path = "download_rzepak/101507_2.0033.187_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [168]:
#małopolskie
cut_few_lots(coord_data_rzepak.iloc[0:5],
            extracted_dir_path = "download_rzepak/121304_5.0013.401_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [170]:
#świętokrzyskie
cut_few_lots(coord_data_rzepak.iloc[5:7],
            extracted_dir_path = "download_rzepak/260704_5.0010.16_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [171]:
#podkarpackie
cut_few_lots(coord_data_rzepak.iloc[7:8],
            extracted_dir_path = "download_rzepak/181004_2.0003.1471_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [173]:
#lubelskie
cut_few_lots(coord_data_rzepak.iloc[8:10],
            extracted_dir_path = "download_rzepak/060103_2.0002.283_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [214]:
#warmińsko_mazurskie
cut_few_lots(coord_data_rzepak.iloc[14:26],
            extracted_dir_path = "download_rzepak/281408_2.0010.116_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [180]:
#dolnośląskie
cut_few_lots(coord_data_rzepak.iloc[22:42],
            extracted_dir_path = "download_rzepak/021804_5.0002.196_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [183]:
#wielkopolskie (brak ostatniej przez brak granic)
cut_few_lots(coord_data_rzepak.iloc[42:51],
            extracted_dir_path = "download_rzepak/300308_2_0015_62_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [185]:
#warmia v2
cut_few_lots(coord_data_rzepak.iloc[52:],
            extracted_dir_path = "download_rzepak/281702_2.0013.79_extracted/",
            out_dir_path = "rzepak/") 

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Nierzepak

In [215]:
os.listdir("download_nierzepak")

['320407_5.0009.4',
 '240506_5.0001.AR_11.41',
 '140601_2.0025.16',
 '200105_2.0025.AR_1.28',
 '221107_2.0008.AR_6.423']

In [220]:
#śląskie (brak granic 1)
cut_few_lots(coord_data_nierzepak.iloc[0:9],
            extracted_dir_path = "download_nierzepak/240506_5.0001.AR_11.41/",
            out_dir_path = "nierzepak/")

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [227]:
#war-maz też brakowało
cut_few_lots(coord_data_nierzepak.iloc[9:15],
            extracted_dir_path = "download_nierzepak/200105_2.0025.AR_1.28/",
            out_dir_path = "nierzepak/")

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [229]:
#mazowieckie
cut_few_lots(coord_data_nierzepak.iloc[15:24],
            extracted_dir_path = "download_nierzepak/140601_2.0025.16/",
            out_dir_path = "nierzepak/")

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [232]:
#pomorskie
cut_few_lots(coord_data_nierzepak.iloc[24:33],
            extracted_dir_path = "download_nierzepak/221107_2.0008.AR_6.423/",
            out_dir_path = "nierzepak/")

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [234]:
#zach-pomorskie
cut_few_lots(coord_data_nierzepak.iloc[33:],
            extracted_dir_path = "download_nierzepak/320407_5.0009.4/",
            out_dir_path = "nierzepak/")

/home/piotr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Summary

In [235]:
#liczba rzepakowych wycietych

In [236]:
len(os.listdir("rzepak"))

93

In [237]:
#liczba nierzepakowych wycietych

In [238]:
len(os.listdir("nierzepak"))

36